In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

tqdm.pandas()

In [2]:
path = '/content/drive/MyDrive/Магистратура/Экспертные и рекомендательные системы/Курсовая работа v2/data/'

interactions_df = pd.read_csv(path+'interactions.csv')
interactions_df.shape

(1533078, 5)

In [3]:
interactions_df.isna().sum()

user_id             0
item_id             0
progress            0
rating        1247722
start_date          0
dtype: int64

In [4]:
interactions_df.fillna(0, inplace=True)

In [5]:
interactions_df['start_date'] = pd.to_datetime(interactions_df['start_date'])
interactions_df['year'] = interactions_df['start_date'].dt.year
interactions_df['month'] = interactions_df['start_date'].dt.month

In [6]:
interactions_df['rating'].value_counts(normalize=True)

0.0    0.813867
5.0    0.140055
4.0    0.026343
3.0    0.011920
1.0    0.003989
2.0    0.003825
Name: rating, dtype: float64

In [7]:
def most_popular_item(interactions_df, year=None, month=None):
  temp_ = interactions_df[['item_id', 'rating', 'year', 'month']].copy()

  if not year and not month:
    temp_ = temp_[['item_id', 'rating']].dropna().groupby('item_id')['rating'].agg(['mean', 'count']).sort_values(by=['mean', 'count'], ascending=False)

  elif year and not month:
    temp_ = temp_[['item_id', 'rating', 'year']]
    temp_ = temp_[temp_['year'] == year]
    temp_ = temp_.dropna().groupby('item_id')['rating'].agg(['mean', 'count']).sort_values(by=['mean', 'count'], ascending=False)

  elif not year and month:
    temp_ = temp_[['item_id', 'rating', 'month']]
    temp_ = temp_[temp_['month'] == month]
    temp_ = temp_.dropna().groupby('item_id')['rating'].agg(['mean', 'count']).sort_values(by=['mean', 'count'], ascending=False)

  else:
    temp_ = temp_[['item_id', 'rating', 'year', 'month']]
    temp_ = temp_[(temp_['month'] == month) & (temp_['year'] == year)]
    temp_ = temp_.dropna().groupby('item_id')['rating'].agg(['mean', 'count']).sort_values(by=['mean', 'count'], ascending=False)

  top = temp_.index[:10]
  return top.values

In [8]:
best_books_time = {}

for year in [None, 2018, 2019]:
  for month in [None] + list(range(1, 13)):
    top = most_popular_item(interactions_df, year, month)
    best_books_time[f"{year}_{month}"] = top

In [65]:
import pickle

with open(path+'best_books_time.pickle', 'wb') as handle:
  pickle.dump(best_books_time, handle)

In [52]:
interactions_df.loc[interactions_df['rating'].isna(), ['user_id', 'item_id']].to_csv(path+'saw_but_didnt_rate.csv', index=False)

In [13]:
interactions_df['sequence'] = interactions_df.groupby('user_id').cumcount() + 1

In [21]:
interactions_df.drop(columns=['progress']).to_csv(path+'interactions_clean.csv', index=False)